In [19]:
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
tf.reset_default_graph()
#training paramaters
learning_rate = 0.001
training_steps = 2000
batch_size = 128
display_step = 200

#network paramters
num_input = 28
timesteps = 28
num_hidden = 128
num_classes = 10

#variables
X = tf.placeholder("float",[None,timesteps,num_input])
Y = tf.placeholder("float",[None,num_classes])

#define weights
w = tf.Variable(tf.random_normal(shape=[num_hidden, num_classes], stddev=0.01), name='weights')
b = tf.Variable(tf.random_normal([num_classes]), name="bias")

def RNN(x,w,b):
    x = tf.unstack(x,timesteps,1)
    lstm_cell = rnn.BasicLSTMCell(num_hidden,forget_bias=1.0)
    outputs,states=rnn.static_rnn(lstm_cell,x,dtype=tf.float32)
    return tf.matmul(outputs[-1],w)+b

logits = RNN(X,w,b)
prediction = tf.nn.softmax(logits)

entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
loss = tf.reduce_mean(entropy) # computes the mean over all the examples in the batch

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs/q5', sess.graph) # to visualize using TensorBoard
    sess.run(tf.global_variables_initializer())
    for i in range(1,training_steps+1):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        #reshape to get batch in format for RNN - IMPORTANT
        batch_x = batch_x.reshape((batch_size,timesteps,num_input))
        sess.run(optimizer, feed_dict={X: batch_x, Y: batch_y})
        if(i%200==0 or i==1):
            loss1,acc = sess.run([loss,accuracy],feed_dict={X:batch_x,Y:batch_y})
            print('Average loss epoch {0}: {1}'.format(i, loss1))

    print("Optimization Finished!")
    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Accuracy:", 
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))
    writer.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Average loss epoch 1: 2.5553553104400635
Average loss epoch 200: 0.5510718822479248
Average loss epoch 400: 0.2515178322792053
Average loss epoch 600: 0.21879467368125916
Average loss epoch 800: 0.21018564701080322
Average loss epoch 1000: 0.12655311822891235
Average loss epoch 1200: 0.16527174413204193
Average loss epoch 1400: 0.12723402678966522
Average loss epoch 1600: 0.049125976860523224
Average loss epoch 1800: 0.0835292786359787
Average loss epoch 2000: 0.0723661482334137
Optimization Finished!
Accuracy: 0.984375
